## Pull FRED data

In [ ]:
import fredapi as fred
import pandas as pd
import os

cli=fred.Fred(api_key=os.getenv("FRED_API_KEY", ''))

# Fetch US Unemployment Rate (series ID: UNRATE)
unemployment = cli.get_series('UNRATE', observation_start='1980-01-01', observation_end='2019-12-31')

unemployment = unemployment.to_frame(name='unemployment_rate')
